# Incremental Vector Storage  and Embedding Modulation

## Installing Dependencies

In [2]:
!pip install langchain faiss-cpu pypdf GitPython openpyxl sentence-transformers transformers llama-cpp-python 

  Obtaining dependency information for GitPython from https://files.pythonhosted.org/packages/f9/94/1877b88fa3a0a30bedb43757a14f548c3b2719c8d83c16012f89564c0f3b/GitPython-3.1.36-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 5.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
  Obtaining dependency information for diskcache>=5.6.1 from https://files.pythonhosted.org/packages/3f/27/4570e78fc0bf5ea0ca45eb1de3818a23787af9b390c0b0a0033a1b8236f9/diskcache-5.6.3-py3-none-any.whl.metadata


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.6-cp311-cp311-macosx_13_0_arm64.whl size=851328 sha256=f55ebaaafa4f1c349a505d93297e5ce75cb8efc038ed9dcec88d2e83c4cc6a75
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/18/f3/e6/e6d374c76db44b5b0451c3a76b3049f29e881819bc43f53d4d
Successfully built llama-cpp-python


In [6]:
from langchain.embeddings import (
    LlamaCppEmbeddings, 
    HuggingFaceEmbeddings, 
    SentenceTransformerEmbeddings
)

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import (
    PyPDFLoader,
    DataFrameLoader,
    GitLoader
  )
import pandas as pd
import nbformat
from nbconvert import PythonExporter
import os

## Text Extraction Function

In [7]:
def get_text_splits(text_file):
  """Function takes in the text data and returns the  
  splits so for further processing can be done."""
  with open(text_file,'r') as txt:
    data = txt.read()

  textSplit = RecursiveCharacterTextSplitter(chunk_size=150,
                                             chunk_overlap=15,
                                             length_function=len)
  doc_list = textSplit.split_text(data)
  return doc_list

## Pdf Text Extraction function

In [10]:
loader = PyPDFLoader("/Users/apple/Documents/TestingEmbbeding/9780723434177.pdf")
pages = loader.load_and_split()

In [12]:
len(pages)

18

In [ ]:
pages[0].page_content


In [30]:

def get_pdf_splits(pdf_file):
  """Function takes in the pdf data and returns the  
  splits so for further processing can be done."""
  
  loader = PyPDFLoader(pdf_file)
  pages = loader.load_and_split()  

  textSplit = RecursiveCharacterTextSplitter(chunk_size=850,
                                             chunk_overlap=200,
                                             length_function=len)
  doc_list = []
  #Pages will be list of pages, so need to modify the loop
  for pg in pages:
    pg_splits = textSplit.split_text(pg.page_content)# here we are giveing each page content with is a text
    doc_list.extend(pg_splits)

  return doc_list

## Excel Text Extraction function

In [17]:
def get_excel_splits(excel_file,target_col,sheet_name):
  trialDF = pd.read_excel(io=excel_file,
                          engine='openpyxl',
                          sheet_name=sheet_name)
  
  df_loader = DataFrameLoader(trialDF,
                              page_content_column=target_col)
  
  excel_docs = df_loader.load()

  return excel_docs
     


## Coma Seperated Values Text Extraction Fucntion

In [18]:
def get_csv_splits(csv_file):
  """Function takes in the csv and returns the  
  splits so for further processing can be done."""
  csvLoader = CSVLoader(csv_file)
  csvdocs = csvLoader.load()
  return csvdocs

## IPYNB Text Extraction Function

In [20]:
def get_ipynb_splits(notebook):
  """Function takes the notebook file,reads the file 
  data as python script, then splits script data directly"""

  with open(notebook) as fh:
    nb = nbformat.reads(fh.read(), nbformat.NO_CONVERT)

  exporter = PythonExporter()
  source, meta = exporter.from_notebook_node(nb)

  #Python file data is in the source variable
  
  textSplit = RecursiveCharacterTextSplitter(chunk_size=150,
                                             chunk_overlap=15,
                                             length_function=len)
  doc_list = textSplit.split_text(source)
  return doc_lis

## Git Hub File Extraction Function

In [22]:
def get_git_files(repo_link, folder_path, file_ext):
  # eg. loading only python files
  git_loader = GitLoader(clone_url=repo_link,
    repo_path=folder_path, 
    file_filter=lambda file_path: file_path.endswith(file_ext))
  #Will take each file individual document
  git_docs = git_loader.load()

  textSplit = RecursiveCharacterTextSplitter(chunk_size=150,
                                             chunk_overlap=15,
                                             length_function=len)
  doc_list = []
  #Pages will be list of pages, so need to modify the loop
  for code in git_docs:
    code_splits = textSplit.split_text(code.page_content)
    doc_list.extend(code_splits)

  return doc_list

## Incrementation of Exsisting Vector 

In [24]:

def embed_index(doc_list, embed_fn, index_store):
  """Function takes in existing vector_store, 
  new doc_list and embedding function that is 
  initialized on appropriate model. Local or online. 
  New embedding is merged with the existing index. If no 
  index given a new one is created"""
  #check whether the doc_list is documents, or text
  try:
    faiss_db = FAISS.from_documents(doc_list, 
                              embed_fn)  
  except Exception as e:
    faiss_db = FAISS.from_texts(doc_list, 
                              embed_fn)
  
  if os.path.exists(index_store):
    local_db = FAISS.load_local(index_store,embed_fn)
    #merging the new embedding with the existing index store
    local_db.merge_from(faiss_db)
    print("Merge completed")
    local_db.save_local(index_store)
    print("Updated index saved")
  else:
    faiss_db.save_local(folder_path=index_store)
    print("New store created...")

In [25]:

def get_docs_length(index_path, embed_fn):
  test_index = FAISS.load_local(index_path,
                              embeddings=embed_fn)
  test_dict = test_index.docstore._dict
  return len(test_dict.values()) 

In [26]:
#testing out the above function with the open source 
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [37]:
BDC_documents = get_pdf_splits("/Users/apple/Desktop/BookAI 2/BDC.pdf")

In [53]:
BharatAI_documents = get_pdf_splits("/Users/apple/Desktop/Protoype/finalppt.pdf")

In [47]:
len(BDC_documents)

587

In [55]:
len(BharatAI_documents)

12

In [45]:
embed_index(doc_list=BDC_documents,embed_fn=embeddings,index_store='new_index')

New store created...


In [56]:
embed_index(doc_list=BharatAI_documents,embed_fn=embeddings,index_store='new_index')

Merge completed
Updated index saved


In [57]:
get_docs_length(index_path='new_index',embed_fn=embeddings)


599

In [58]:

test_index = FAISS.load_local("new_index",embeddings)

In [60]:
 result = test_index.similarity_search("What is bharat ai")
 result[0]

Document(page_content='SRM’s Bharat AI Navigating Mining Laws with Ease', metadata={})